# Install Library

In [1]:
!pip install ultralytics -q

In [2]:
import os,os.path as osp
from glob import glob
from tqdm import tqdm
import shutil
import random

import cv2
import numpy as np

from ultralytics import YOLO
from ultralytics import RTDETR
from ultralytics import settings

import yaml
from sklearn.model_selection import train_test_split

settings.update({"wandb": False})

# Data preprocessing

In [3]:
dataset_dir = '/kaggle/input/th-road-safety/TrafficHackathon'
train_dir = dataset_dir + '/train'
test_dir = dataset_dir + '/test'
save_dir = '/kaggle/working/yolov8'
val_ratio = 0.05

# Create folder dataset for yolov8
os.makedirs(save_dir,exist_ok=True)
os.makedirs(save_dir + '/' + 'images',exist_ok=True)
os.makedirs(save_dir + '/' + 'labels',exist_ok=True)

os.makedirs(save_dir + '/' + 'images/train',exist_ok=True)
os.makedirs(save_dir + '/' + 'labels/train',exist_ok=True)

os.makedirs(save_dir + '/' + 'images/val',exist_ok=True)
os.makedirs(save_dir + '/' + 'labels/val',exist_ok=True)

# List annotation file  
ann_paths = glob(osp.join(train_dir , '*.txt'))
ann_train, ann_val = train_test_split(ann_paths, test_size=val_ratio)

# Copy train images and labels folder
print('Copy images and labels in train folder')
for ann_path in tqdm(ann_train):
    filename = osp.split(ann_path[:-4])[-1]
    
    img_path = ann_path[0:-4] + '.jpg'
    save_img_path = save_dir + '/images/train/' + filename + '.jpg'
    save_label_path = save_dir + '/labels/train/' + filename + '.txt'
    
    if os.path.exists(img_path):  
        shutil.copy(img_path, save_img_path)
        shutil.copy(ann_path, save_label_path)

# Copy val images and labels folder
print('Copy images and labels in val folder')
for ann_path in tqdm(ann_val):
    filename = osp.split(ann_path[:-4])[-1]
    
    img_path = ann_path[0:-4] + '.jpg'
    save_img_path = save_dir + '/images/val/' + filename + '.jpg'
    save_label_path = save_dir + '/labels/val/' + filename + '.txt'
    
    if os.path.exists(img_path):  
        shutil.copy(img_path, save_img_path)
        shutil.copy(ann_path,  save_label_path)
    
# Create .yaml yolo format
print('Create config file dataset.yaml')
classes_list = []
index = 0

for label in open(train_dir + "/classes.txt", "r").read().split('\n'): 
    classes_list.append(str(index) + ': ' + label)
    index += 1
    
data = {
    "path" : save_dir,
    "train" : save_dir + '/' + 'images/train',
    "val" : save_dir + '/' + 'images/val',
    "names" : classes_list
}

with open('yolov8/dataset.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False, sort_keys=False)

Copy images and labels in train folder


100%|██████████| 1525/1525 [00:23<00:00, 63.65it/s]


Copy images and labels in val folder


100%|██████████| 81/81 [00:01<00:00, 66.05it/s]

Create config file dataset.yaml


# Training

In [11]:
#model = YOLO('yolov8m.pt')  # load a pretrained model (recommended for training)
model = RTDETR("rtdetr-l.pt")
results = model.train(data='yolov8/dataset.yaml',project='detect',name = 'train', 
                     epochs=50
                     )

Ultralytics YOLOv8.2.49 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=rtdetr-l.pt, data=yolov8/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=detect, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sh

train: Scanning /kaggle/working/yolov8/labels/train... 1598 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1598/1598 [00:01<00:00, 1243.45it/s]


train: New cache created: /kaggle/working/yolov8/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /kaggle/working/yolov8/labels/val... 1035 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1035/1035 [00:00<00:00, 1078.43it/s]


val: New cache created: /kaggle/working/yolov8/labels/val.cache
Plotting labels to detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000625, momentum=0.9) with parameter groups 143 weight(decay=0.0), 206 weight(decay=0.0005), 226 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to detect/train2
Starting training for 50 epochs...

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


...

                   all       1035      12728      0.978      0.943      0.968      0.795



50 epochs completed in 2.199 hours.
Optimizer stripped from detect/train2/weights/last.pt, 66.2MB
Optimizer stripped from detect/train2/weights/best.pt, 66.2MB

Validating detect/train2/weights/best.pt...
Ultralytics YOLOv8.2.49 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
rt-detr-l summary: 494 layers, 32008400 parameters, 0 gradients, 103.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:27<00:00,  1.22it/s]


                   all       1035      12728      0.978      0.944      0.968      0.796
        0: pedestrian         698       1368      0.968      0.936      0.972      0.726
            1: bicycle        191        223      0.997      0.933       0.96      0.804
         2: motorcycle        809       4621      0.959       0.92      0.958      0.763
                3: car        893       3548      0.963      0.907      0.961      0.766
               4: taxi        526        812      0.956      0.941      0.971      0.816
                5: van        169        189      0.989       0.95      0.974      0.759
              6: truck        651       1370      0.988      0.966      0.983      0.831
                7: bus         98        136      0.985      0.955      0.955      0.787
      8: emergency_car         20         20      0.984          1      0.995      0.931
             9: tuktuk        173        323      0.966      0.882      0.921      0.729
             10: othe

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [4]:
import torch
torch.cuda.empty_cache()

In [ ]:
model = RTDETR("rtdetr-x.pt")
results = model.train(data='yolov8/dataset.yaml',project='detect',name = 'train', 
                     epochs=50,
                    batch=8
                     #auto_augment='autoaugment'
                     )
#                      hsv_h=0.1,
#                      hsv_s=0.3,
#                      hsv_v=0.3,
#                      flipud=0.1,
#                      fliplr=0.5,
#                      mosaic=0.5,
#                      mixup=0.2

100%|██████████| 129M/129M [00:00<00:00, 294MB/s] 


Ultralytics YOLOv8.2.50 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=rtdetr-x.pt, data=yolov8/dataset.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=detect, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show

100%|██████████| 755k/755k [00:00<00:00, 26.0MB/s]
2024-07-06 16:41:10,277	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-07-06 16:41:11,365	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=12
WARNING ⚠️ no model scale passed. Assuming scale='x'.

                   from  n    params  module                                       arguments                     
  0                  -1  1     25792  ultralytics.nn.modules.block.HGStem          [3, 32, 64]                   
  1                  -1  6    259200  ultralytics.nn.modules.block.HGBlock         [64, 64, 128, 3, 6]           
  2                  -1  1      1408  ultralytics.nn.modules.conv.DWConv           [128, 128, 3, 2, 1, False]    
  3                  -1  6   1248256  ultralytics.nn.modules.block.HGBlock         [128, 128, 512, 3, 6]         
  4                  -1  6   1788928  ultralytics.nn.modules.block.HGBlock         [512, 128, 512, 3, 6, False, True]
  5                  -1  1      5632  ultralytics.nn.modules.conv.DWConv           [512, 512, 3, 2, 1, False]    
  6                  -1  6   2079232  ultralytics.nn.modules.block.HGBlock         [512, 256, 1024, 5, 6

100%|██████████| 6.25M/6.25M [00:00<00:00, 122MB/s]


...


      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


  0%|          | 0/191 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/autograd/__init__.py:251: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/Context.cpp:71.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
       7/50      10.2G     0.4654     0.4453    0.05901         68        640: 100%|██████████| 191/191 [03:20<00:00,  1.05s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:02<00:00,  2.01it/s]

                   all         81        904      0.936       0.77      0.842      0.605



      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


  0%|          | 0/191 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/autograd/__init__.py:251: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/Context.cpp:71.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
       8/50      10.3G     0.4454     0.4251    0.05969         29        640: 100%|██████████| 191/191 [03:21<00:00,  1.05s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/6 [00:00<?, ?it/s]

# Inference

In [ ]:
answer_list = []
# Use the model
model = RTDETR('/kaggle/working/detect/train/weights/best.pt')

for file in tqdm(glob(test_dir + '/*')):
    
    bbox_list = []
    cls_list = []
    scores_list = []

    # Predict on an image
    results = model(file, verbose=False)
    # Process results list
    for result in results:
        
        boxes = result.boxes
        for box in boxes:
            coor_box = box.xyxy.cpu().numpy().tolist()  # Boxes object for bounding box outputs
            class_box = box.cls.cpu().numpy().tolist()
            class_scores = box.conf.cpu().numpy().tolist()
        
            bbox_list.append(coor_box[0])
            cls_list.append(int(class_box[0]))
            scores_list.append(class_scores[0])
        
        value = (file.split('/')[-1], bbox_list, cls_list,scores_list )

    answer_list.append(value)

In [ ]:
import pandas as pd
column_name = ['id','boxes', 'labels', 'scores']
xml_df = pd.DataFrame(answer_list, columns=column_name)
xml_df.to_csv('/kaggle/working/50_RTDTRx_finetune.csv', index=None)

In [18]:
xml_df

,id,boxes,labels,scores
0,TF7-KS-2-03-28032023_1411.jpg,"[[1604.2265625, 772.9917602539062, 1782.855346...","[2, 3, 2, 2, 3, 6]","[0.9686948657035828, 0.963720977306366, 0.9288..."
1,TF7-DM-1-03-28032023(12.00-13.00)_1563.jpg,"[[620.3967895507812, 126.69283294677734, 985.8...","[3, 4, 2, 3, 3, 2, 2, 2]","[0.9757187366485596, 0.9729746580123901, 0.978..."
2,TF7-DM-02-28032023(12.00-13.00)_964.jpg,"[[205.878173828125, 827.057373046875, 580.6764...","[3, 4, 3, 3, 2, 2, 2, 3, 2, 2, 2, 2, 4, 2, 1, ...","[0.9816930294036865, 0.9542747139930725, 0.931..."
3,TF7-BT-3-03-28032023(12.00-13.00)_1338.jpg,"[[1494.9986572265625, -0.06147220730781555, 17...","[3, 2, 2]","[0.9589695334434509, 0.879623532295227, 0.8697..."
4,TF7-KS-1-01-28032023(12.00-13.00)_1444.jpg,"[[1109.875244140625, 159.33265686035156, 1170....","[6, 6, 6, 0, 6]","[0.4902559518814087, 0.3028828501701355, 0.507..."
...,...,...,...,...
1402,TF7-KS-1-02-28032023(12.00-13.00)_1263.jpg,"[[890.6900024414062, 503.99090576171875, 1118....","[3, 3, 0, 1, 2, 2, 2, 7, 2, 3, 2, 2, 2, 2, 2, ...","[0.9504495859146118, 0.9273397922515869, 0.959..."
1403,TF7-RW-1-03-28032023(12.00-13.00)_859.jpg,"[[975.640380859375, 251.21788024902344, 1301.8...","[3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4]","[0.9795659184455872, 0.9412915706634521, 0.969..."
1404,TF7-KS-1-03-28032023(12.00-13.00)_984.jpg,"[[1055.2891845703125, 233.0370330810547, 1358....","[3, 6, 3, 4, 2]","[0.9763023853302002, 0.9730518460273743, 0.984..."
1405,TF7-KS-1-01-28032023(12.00-13.00)_1234.jpg,"[[1131.261962890625, 225.55148315429688, 1221....","[3, 6, 2, 4, 2, 0, 2, 3, 3]","[0.9173907041549683, 0.8902944326400757, 0.830..."
